In [6]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix, csc_matrix, issparse, vstack
from pandas.api.types import CategoricalDtype
from IPython.display import clear_output
import timeit, json, warnings
from tqdm import notebook

# Prediction
----

In [2]:
df_s = pd.read_csv('../data/recommendation/freq_rating_item_recommendation.csv')
s_width = int(df_s.shape[1] / 2) # size of similarity values
s_products = df_s.iloc[:, 1:s_width+1]
sim_product = df_s.iloc[:, -s_width:].to_numpy()

products = json.load(open('../data/interaction/products/freq_rating_item_products.json'))

sim = pickle.load(open('../data/similarity/freq_rating_item_similarity.pkl', 'rb'))

df_s

,Recommendation for product:,Match 1.,Match 2.,Match 3.,Match 4.,Match 5.,Match 6.,Match 7.,Match 8.,Match 9.,...,Similarity 6.,Similarity 7.,Similarity 8.,Similarity 9.,Similarity 10.,Similarity 11.,Similarity 12.,Similarity 13.,Similarity 14.,Similarity 15.
0,#2 Coffee Filters,Compostable Coffee Filters,Organic Hass Avocado,Organic Garlic,Bag of Organic Bananas,Organic Yellow Onion,Major Dickason's Blend Ground Coffee Dark Roast,Organic Strawberries,Organic Lemon,Organic Baby Spinach,...,0.049973,0.048253,0.047569,0.047076,0.046623,0.046103,0.045945,0.044290,0.043183,0.043164
1,0% Fat Blueberry Greek Yogurt,Greek 0% Fat Strawberry on the Bottom Yogurt,0% Fat Superfruits Greek Yogurt,0% Fat Organic Greek Vanilla Yogurt,"0% Greek, Blueberry on the Bottom Yogurt",Nonfat Strawberry With Fruit On The Bottom Gre...,"Organic Nonfat Yogurt, Peach",YoKids Strawberry Banana Organic Lowfat Yogurt,Vegetable Dumpling,Layered Dark Chocolate Coconut Granola Bars,...,0.039299,0.037897,0.035791,0.032983,0.032233,0.032127,0.031653,0.031442,0.030699,0.030687
2,0% Fat Free Organic Milk,Vitamin D Organic Whole Milk,Organic Skim Milk,Organic 1% Milk,2% Reduced Fat Organic Milk,Banana,Organic Skim Milk with DHA Omega-3,Large Lemon,Organic Large Grade A Brown Eggs,Light Multi-Grain English Muffins,...,0.063104,0.059423,0.055346,0.054124,0.053103,0.051740,0.049951,0.048660,0.047975,0.047058
3,0% Fat Organic Greek Vanilla Yogurt,Vanilla Greek Yogurt 0% Fat,0% Fat Blueberry Greek Yogurt,Greek 0% Fat Strawberry on the Bottom Yogurt,Organic Vanilla Bean Greek Yogurt,YoKids Strawberry Banana Organic Lowfat Yogurt,0% Fat Superfruits Greek Yogurt,Organic Strawberries,Nonfat Plain Greek Yogurt,Organic Baby Spinach,...,0.077823,0.056596,0.053974,0.051517,0.046818,0.046076,0.045680,0.045299,0.045136,0.043073
4,0% Fat Superfruits Greek Yogurt,Greek 0% Fat Strawberry on the Bottom Yogurt,0% Fat Blueberry Greek Yogurt,0% Fat Organic Greek Vanilla Yogurt,Nonfat Strawberry With Fruit On The Bottom Gre...,YoKids Strawberry Banana Organic Lowfat Yogurt,"0% Greek, Blueberry on the Bottom Yogurt",Cherry Pomegranate Greek Yogurt,Vanilla Greek Yogurt 0% Fat,Organic Greek Nonfat Yogurt With Mixed Berries,...,0.060427,0.058599,0.058476,0.057264,0.052049,0.048396,0.043193,0.042597,0.042537,0.042423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,of Hanover 100 Calorie Pretzels Mini,100 Calorie Pretzels Packs,Goldfish Cheddar Baked Snack Crackers Multi Packs,Cheez-It Packs,Classic Mix Variety,Variety Pack Snack Stacks,Pretzel Snaps Pretzel Packs,100 Calorie Per Bag Popcorn,Oreo Mini/Nutter Butter Bites/Mini Chips Ahoy!...,Original/Ranch/BBQ/Chile Picante con Limon Var...,...,0.111210,0.110836,0.104913,0.103571,0.101299,0.099143,0.094165,0.090169,0.089389,0.088879
9932,smartwater® Electrolyte Enhanced Water,Smartwater Electrolyte Enhanced Water,Electrolyte Enhanced Water,Sensitive Toilet Paper,Ultra Soft Facial Tissues,Cola Soft Drink,Original Glass Cleaner,Classic Hummus,Banana,Sparkling Natural Mineral Water,...,0.060350,0.059629,0.057630,0.057304,0.057138,0.055620,0.055537,0.054315,0.053404,0.052331
9933,vitaminwater® XXX Acai Blueberry Pomegranate,Power-C Dragonfruit Vitamin Water Drink,Vitaminwater Revive Fruit Punch,Energy Tropical Citrus Vitamin Water Drink,Vitaminwater® Focus Kiwi-Strawberry,Essential Orange-Orange Vitamin Water Drink,VitaminWater Zero™ XXX Acai Blueberry Pomegranate,Vitamin Water Zero Squeezed Lemonade,Zero Vitamin Water,Vitamin Water Zero Rise Orange,...,0.102554,0.078562,0.077014,0.074649,0.070355,0.060045,0.048015,0.043550,0.042980,0.042882
9934,with Crispy Almonds Cereal,Cherrios Honey Nut,Cinnamon Toast Crunch,Raisin Bran,Honey Bunches of Oats Roasted Oats,0% Greek Strained Yogurt,Clementines,Trail Mix,Cheez-It Cheddar Cracker,Crunchy Oats 'n Honey Granola Bars,...,0.200865,0.196640,0.167157,0.165352,0.162647,0.159786,0.158736,0.155532,0.155194,0.154455


In [3]:
s_ix = s_products.replace(products.values(), products.keys())
s_ix_np = s_ix.to_numpy()
s_ix

,Match 1.,Match 2.,Match 3.,Match 4.,Match 5.,Match 6.,Match 7.,Match 8.,Match 9.,Match 10.,Match 11.,Match 12.,Match 13.,Match 14.,Match 15.
0,1869,5847,5735,580,6698,4443,6456,5957,5301,6674,1190,3544,6184,4026,6826
1,3422,4,3,7,5058,6112,9867,9489,4118,5804,6456,7868,1324,8315,9885
2,9562,6356,5209,144,628,6357,4095,5947,4238,8640,1107,6668,2952,8060,5363
3,9396,1,3422,6594,9867,4,6456,5056,5301,5285,5363,5291,628,4095,5789
4,3422,1,3,5058,9867,7,1324,9396,5804,2688,9065,7953,6456,6109,5802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,22,3331,1308,1677,9441,7441,21,5198,6957,591,4023,603,4460,8974,5197
9932,8289,2525,8082,9227,1828,6817,1664,628,8454,4877,5151,9659,2943,2952,6951
9933,7384,9567,2533,9570,2557,9566,9565,9918,9564,9910,7783,9835,6456,628,580
9934,1312,1631,7603,3735,5,1735,9125,1307,2087,6354,9707,7491,440,4646,7734


In [4]:
def predict(R, method):
    predictions = None
    
    if method == 'rating' or method == 'count':
        batchsize = 10000 # tests have shown that this is a good batch size to avoid performance issues
        df_s = pd.read_csv('../data/recommendation/freq_rating_item_recommendation.csv')
        s_width = int(df_s.shape[1] / 2) # size of similarity values
        sim_product = df_s.iloc[:, -s_width:].to_numpy()
        
        # handle in class as instance variable for performance reasons and check if issparse(S) and k nearest is the same
        col_ix = np.array([s_ix_np.shape[1] * [i] for i in range(s_ix_np.shape[0])]).ravel()
        row_ix = s_ix_np.astype(int).ravel()
        A = np.zeros(sim.shape)
        A[row_ix, col_ix] = 1
        S = A * sim # hadamard product to just keep k similarities
        S = csr_matrix(S)
        
        # convert R to sparse column matrix if not already done
        if isinstance(R, pd.DataFrame):
            R = csc_matrix(R.values)
        elif isinstance(R, np.matrix) or isinstance(R, np.ndarray):
            R = csc_matrix(R)
        elif issparse(R):
            pass
        
        # perform batchwise predictions
        i_prev = 0
        denominators = 1 / np.sum(np.absolute(sim_product), axis = 1)
        
        for i in range(batchsize, R.shape[0] + batchsize, batchsize):
            # batch prep
            i = min(i, R.shape[0])
            print('i_prev: ', i_prev, '\ti: ', i)
            batch = R[i_prev:i]
            
            # numerators
            batch_predictions = batch.dot(S)

            # denominators with hadamard product
            D = np.array([[denominator] * batch_predictions.shape[0] for denominator in denominators]).T
            batch_predictions = batch_predictions.multiply(D)
            
            # append batch to predictions
            if issparse(predictions):
                predictions = vstack([predictions, batch_predictions])
            else:
                predictions = batch_predictions
            
            #update slice index
            i_prev = i
        
    elif method == 'binary':
        pass
    
    return predictions

In [7]:
R = pickle.load(open('../data/interaction/freq_rating_item_interaction.pkl', 'rb'))
predict(R, 'rating')

i_prev:  0 	i:  10000
i_prev:  10000 	i:  20000
i_prev:  20000 	i:  30000
i_prev:  30000 	i:  40000
i_prev:  40000 	i:  50000
i_prev:  50000 	i:  60000
i_prev:  60000 	i:  70000
i_prev:  70000 	i:  80000
i_prev:  80000 	i:  90000
i_prev:  90000 	i:  100000
i_prev:  100000 	i:  110000
i_prev:  110000 	i:  120000
i_prev:  120000 	i:  130000
i_prev:  130000 	i:  140000
i_prev:  140000 	i:  150000
i_prev:  150000 	i:  160000
i_prev:  160000 	i:  170000
i_prev:  170000 	i:  174052


<174052x9936 sparse matrix of type '<class 'numpy.float64'>'
	with 694334122 stored elements in COOrdinate format>